<a href="https://colab.research.google.com/github/VLADISLAV008/DSL/blob/main/Job3/Job%233.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Implementation of FIRST and FOLLOW functions

We are given a context free grammar $G(\Sigma, N, S \in N, P)$

We assume that in our grammar there is no useless nonterminal symbols and left recurtion!

To start with, we will use the following context free grammar structure:
```
{'toks': set(token), 'vars': dict(var: definition), 'hvar': var}
token: (class, value)
class: int
value: str
var: str                 # name of the non-terminal symbol
definition: list(rule)
rule: list(var | token)  # right part of the rule
```

Let's implement a method that returns all nullable (vanishing) symbols.

A nonterminal symbol $X$ is called nullable, if $X \twoheadrightarrow e$.

We will expand the set of nullable symbols as it grows.

$V_0 = \emptyset$

$V_1 = \{X \in N | \space (X \to e) \in P\}$
$V_2 = \{X \in N | \space \exists k \in \mathbb{N} \space \exists B_1, B_2, ..., B_k \in V_1, (X \to B_1 B_2 ... B_k) \in P\}$

...

$V_{n+1} = \{X \in N | \space \exists k \in \mathbb{N} \space \exists B_1, B_2, ..., B_k \in V_n, (X \to B_1 B_2 ... B_k) \in P\}$

$V_n = V_{n+1} \supset N$ - the desired set of nullable symbols.

In [1]:
def nullable_symbols(grammar) -> set:
  nullable = set()
  prev_count = None

  def all_nullable_symbols(rule: str) -> bool:
    return all(map(lambda s: s in nullable or s == (0, ''), rule))

  while len(nullable) != prev_count:
    prev_count = len(nullable)
    nullable_symbols = [var for var, definition in grammar['vars'].items() 
                        if list(filter(all_nullable_symbols, definition)) != []]
    nullable = nullable.union(set(nullable_symbols))

  return nullable  

### **Definition**

$ \forall A \in N: FIRST(A) = a \in 𝛴 ∣ A ↠ aα$. 

If $A$ is the nullable nonterminal symbol, then the end of input $⊳$ is added to $FIRST(A)$.


In [2]:
def first(grammar, nonterminal: str) -> set:
  toks = set()
  if nonterminal in nullable_symbols(grammar):
    toks.add((0, ''))

  for rule in grammar['vars'][nonterminal]:
    if rule[0] in grammar['toks']:
      toks.add(rule[0])
    else:   
      toks.update(first(grammar, rule[0]))
    
  return toks

### **Definition**

$ \forall A \in N: FOLLOW(A) = a \in 𝛴 ∣ S↠𝛼Aa𝛽$. 

If $S↠𝛼A$, then the end of input $⊳$ is added to $FOLLOW(A)$.

In [3]:
def follow(grammar, nonterminal: str) -> set:
  toks = set()

  return toks

In [4]:
def test():
  grammar = {
      'toks': {(0, ''), (1, 'a'), (2, 'b'), (3, 'c')},
      'vars': {
          'S': [[(1, 'a'), 'M'],
                [(1, 'a'), 'S'],
                ['K']], 
          'K': [[(3, 'c')], 
                [(0, '')]], 
          'M': [['M', 'P']],
          'P': [[(3, 'c')]]
          },
      'hvar': 'S'
      }

  print(grammar)

  print('Nullable symbols are ' + str(nullable_symbols(grammar)))

test()  

{'toks': {(3, 'c'), (2, 'b'), (0, ''), (1, 'a')}, 'vars': {'S': [[(1, 'a'), 'M'], [(1, 'a'), 'S'], ['K']], 'K': [[(3, 'c')], [(0, '')]], 'M': [['M', 'P']], 'P': [[(3, 'c')]]}, 'hvar': 'S'}
Nullable symbols are {'K', 'S'}
